In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn import linear_model
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics.pairwise import sigmoid_kernel, rbf_kernel, polynomial_kernel
from sklearn import neighbors
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


## 1. Load Data

In [2]:
# Load the features
X = np.loadtxt(open("AD_MRI_DATA/2-CN_MCI/2.1-MRI_CorticalThickness.csv","rb"),delimiter=",",skiprows=0)
X = np.array(X, dtype=float)
print("X:" + str(X.shape))

# Load the labels
with open('AD_MRI_DATA/2-CN_MCI/2-Phenotype.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    c1 = [row[1]for row in reader]
    
del c1[0]
y = np.array(c1, dtype=int)
print("y:" + str(y.shape))


X:(188, 136)
y:(188,)


## 2. Lasso with Classifiers

### 2.1 Lasso

In [3]:
model = linear_model.LassoCV(alphas=[0.001,0.002,0.0001])
#alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(lasso.coef_)
3
selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

alpha: 0.002
X_selected: (188, 50)


/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03420186089995614, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.004417121517443334, tolerance: 0.0037259999999999984
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012114528262732183, tolerance: 0.0037393333333333345
  model = cd_fast.enet_coordinate_descent_gram(


### 2.2 Classifiers

In [5]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC(C=1, random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    # Model Evaluation
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
    print("auc: "+str(auc_avg)+"\n")


SVM
[[80 20]
 [69 19]]
              precision    recall  f1-score   support

           0       0.49      0.22      0.30        88
           1       0.54      0.80      0.64       100

    accuracy                           0.53       188
   macro avg       0.51      0.51      0.47       188
weighted avg       0.51      0.53      0.48       188

auc: 0.6208823529411764

KNN
[[31 69]
 [18 70]]
              precision    recall  f1-score   support

           0       0.50      0.80      0.62        88
           1       0.63      0.31      0.42       100

    accuracy                           0.54       188
   macro avg       0.57      0.55      0.52       188
weighted avg       0.57      0.54      0.51       188

auc: 0.5580555555555555

Gaussian Naive Bayes
[[48 52]
 [30 58]]
              precision    recall  f1-score   support

           0       0.53      0.66      0.59        88
           1       0.62      0.48      0.54       100

    accuracy                           0.56   

### 2.3 Multi-core SVM

In [6]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = (acc[0] + acc[1] + acc[2])/3
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.0 w2:0.1 w3:0.9
score:0.6578947368421053


In [7]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[66 34]
 [37 51]]
              precision    recall  f1-score   support

           0       0.60      0.58      0.59        88
           1       0.64      0.66      0.65       100

    accuracy                           0.62       188
   macro avg       0.62      0.62      0.62       188
weighted avg       0.62      0.62      0.62       188

auc: 0.6873856209150326



## 3. ElasticNet with Classifiers

### 3.1 ElasticNet

In [8]:
model = ElasticNetCV(random_state=0,alphas=[1,0.1,0.01,0.005,0.001,0.0001])
#,alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(els.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

alpha: 0.01
X_selected: (188, 16)


/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09419039890470682, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.011756252579663062, tolerance: 0.0037259999999999984
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.018462342891623962, tolerance: 0.0037393333333333345
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/op

### 3.2 Classifiers

In [9]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC(C=1, random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    # Model Evaluation
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
    print("auc: "+str(auc_avg)+"\n")

SVM
[[67 33]
 [39 49]]
              precision    recall  f1-score   support

           0       0.60      0.56      0.58        88
           1       0.63      0.67      0.65       100

    accuracy                           0.62       188
   macro avg       0.61      0.61      0.61       188
weighted avg       0.62      0.62      0.62       188

auc: 0.6890522875816993

KNN
[[28 72]
 [15 73]]
              precision    recall  f1-score   support

           0       0.50      0.83      0.63        88
           1       0.65      0.28      0.39       100

    accuracy                           0.54       188
   macro avg       0.58      0.55      0.51       188
weighted avg       0.58      0.54      0.50       188

auc: 0.563937908496732

Gaussian Naive Bayes
[[56 44]
 [31 57]]
              precision    recall  f1-score   support

           0       0.56      0.65      0.60        88
           1       0.64      0.56      0.60       100

    accuracy                           0.60    

### 3.3 Multi-core SVM

In [10]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = (acc[0] + acc[1] + acc[2])/3
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.0 w2:0.9 w3:0.09999999999999998
score:0.7368421052631579


In [11]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[72 28]
 [31 57]]
              precision    recall  f1-score   support

           0       0.67      0.65      0.66        88
           1       0.70      0.72      0.71       100

    accuracy                           0.69       188
   macro avg       0.68      0.68      0.68       188
weighted avg       0.69      0.69      0.69       188

auc: 0.7203921568627452

